In [1]:
import numpy as np
import glob
import cv2
import tensorflow as tf
import random
np.random.seed(123)

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, concatenate
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
#from keras.backend.tensorflow_backend import set_session

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = "D:\\Work\\Signature Verification\\data\\OfflineSignatures\\"
chinese_path = "Chinese\\TrainingSet\\"
dutch_path = "Dutch\\TrainingSet\\"
forged_path = "Offline Forgeries\\"
genuine_path = "Offline Genuine\\"
genuine_sig = {}
forged_sig = {}
batch_size = 32
'''config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))'''

'config = tf.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.9\nset_session(tf.Session(config=config))'

In [20]:
def load_data(path, X=[]):
    images = glob.glob(path+"*.png")
    sig_dict = {}
    for image in images:
        name = image.split("\\")[-1].split("_")[0]
        name = name[-3:]
        temp_image = cv2.imread(image)
        temp_image = cv2.resize(temp_image, (250, 100))
        if name not in sig_dict:
            sig_dict[name] = []
        sig_dict[name].append(temp_image)
    return sig_dict

In [4]:
genuine_sig = load_data(path+chinese_path+genuine_path)

In [5]:
forged_sig = load_data(path+chinese_path+forged_path)

In [6]:
print(forged_sig["001"][1].shape)

(100, 250, 3)


In [7]:
in_dims = (100, 250, 3)
out_dims = 128

In [8]:
def triplet_loss(y_true, y_pred, alpha=0.3):
    anchor_en, positive_en, negative_en = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor_en, positive_en)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor_en, negative_en)), axis=-1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))

    return loss

In [9]:
model = Sequential()
model.add(Conv2D(32, (5, 11), activation="relu", input_shape=in_dims))
#print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 3)))
#print(model.output_shape)
model.add(Conv2D(64, (3, 5), activation="relu", input_shape=in_dims))
#print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2)))
#print(model.output_shape)
model.add(Dropout(0.25))
model.add(Flatten())
#print(model.output_shape)
model.add(Dense(512, activation="relu"))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(out_dims))
model.count_params()

28741504

In [10]:
anchor_ex = Input(shape=in_dims)
positive_ex = Input(shape=in_dims)
negative_ex = Input(shape=in_dims)

anchor_en = model(anchor_ex)
positive_en = model(positive_ex)
negative_en = model(negative_ex)
#merged = concatenate([anchor_en, positive_en, negative_en], axis=-1)

In [11]:
model = Model(inputs=[anchor_ex, positive_ex, negative_ex], outputs=[anchor_en, positive_en, negative_en])
model.compile(loss=triplet_loss, optimizer='adam')

In [12]:
sub_key_s = "0"
sub_key_d = "00"
X_batch = []
for i in range(1, len(genuine_sig.keys())+1):
    sub_key1 = str(i)
    if len(sub_key1) == 1:
        sub_key = sub_key_d+sub_key1
    else:
        sub_key = sub_key_s+sub_key1
    for j in range(len(genuine_sig[sub_key])):
        anchor_img = genuine_sig[sub_key][j]
        for k in range(len(genuine_sig[sub_key])):
            if k!=j:
                positive_img = genuine_sig[sub_key][k]
                for l in range(len(forged_sig[sub_key])):
                    negative_img = forged_sig[sub_key][l]
                    X_batch.append([anchor_img, positive_img, negative_img])
                    if len(X_batch)==batch_size:
                        X_batch = np.array(X_batch)
                        X_batch = X_batch.astype('float32')/255
                        Y_batch = [0]*batch_size
                        Y_batch = np.array(Y_batch)
                        model.fit([X_batch[:, 0], X_batch[:, 1], X_batch[:, 2]], [Y_batch, Y_batch, Y_batch], batch_size=batch_size, epochs=1)
                        X_batch = []

Epoch 1/1
32/32 [==============================] - 4s 122ms/step - loss: 9.0457 - sequential_1_loss: 6.5902
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 37.7895 - sequential_1_loss: 0.0000e+00
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - sequential_1_loss: 0.0000e+00
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 82.7158 - sequential_1_loss: 14.0384
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - sequential_1_loss: 0.0000e+00
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 30.9266 - sequential_1_loss: 24.9713
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 3.5089 - sequential_1_loss: 1.4434
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 21.0735 - sequential_1_loss: 3.6292
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.6071 - sequential_1_loss: 0.0000e+00
Epoch 1/1
32/3

Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8997 - sequential_1_loss: 0.3005
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9006 - sequential_1_loss: 0.3011
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8988 - sequential_1_loss: 0.2999
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8995 - sequential_1_loss: 0.3014
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9000 - sequential_1_loss: 0.3009
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9010 - sequential_1_loss: 0.3003
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9018 - sequential_1_loss: 0.3002
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8986 - sequential_1_loss: 0.2992
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8987 - sequential_1_loss: 0.3005
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9014 - sequential_1_loss: 0.3002
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9018 - sequential_1_loss: 0.2997
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9002 - sequential_1_loss: 0.3001
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9016 - sequential_1_loss: 0.3008
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9010 - sequential_1_loss: 0.3007
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8995 - sequential_1_loss: 0.3001
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9015 - sequential_1_loss: 0.2988
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9007 - sequential_1_loss: 0.3000
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9016 - sequential_1_loss: 0.3003
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8998 - sequential_1_loss: 0.3003
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9022 - sequential_1_loss: 0.3024
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8989 - sequential_1_loss: 0.3008
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8982 - sequential_1_loss: 0.2973
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9005 - sequential_1_loss: 0.2992
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9000 - sequential_1_loss: 0.3000
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9001 - sequential_1_loss: 0.3010
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8999 - sequential_1_loss: 0.2994
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9013 - sequential_1_loss: 0.3015
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9015 - sequential_1_loss: 0.3012
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9050 - sequential_1_loss: 0.3038
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8986 - sequential_1_loss: 0.3004
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8981 - sequential_1_loss: 0.2990
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9040 - sequential_1_loss: 0.3018
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9020 - sequential_1_loss: 0.3017
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9009 - sequential_1_loss: 0.3006
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8979 - sequential_1_loss: 0.2998
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8986 - sequential_1_loss: 0.2977
Epoch 1/1
32/32 [=============================

...

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.2228 - sequential_1_loss: 0.2374
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9171 - sequential_1_loss: 0.1886
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8754 - sequential_1_loss: 0.3246
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.1500 - sequential_1_loss: 0.3666
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.0247 - sequential_1_loss: 0.3589
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8929 - sequential_1_loss: 0.3266
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.7459 - sequential_1_loss: 0.3949
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9899 - sequential_1_loss: 0.1841
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9355 - sequential_1_loss: 0.3383
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.7883 - sequential_1_loss: 0.3099
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.2623 - sequential_1_loss: 0.3404
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.0962 - sequential_1_loss: 0.3028
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9360 - sequential_1_loss: 0.4249
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8033 - sequential_1_loss: 0.3313
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9549 - sequential_1_loss: 0.2389
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.8490 - sequential_1_loss: 0.2213
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.2450 - sequential_1_loss: 0.3951
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.0297 - sequential_1_loss: 0.4293
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.2043 - sequential_1_loss: 0.4919
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.7663 - sequential_1_loss: 0.3081
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9155 - sequential_1_loss: 0.2713
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.3216 - sequential_1_loss: 0.3443
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.3516 - sequential_1_loss: 0.1443
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.5035 - sequential_1_loss: 0.1701
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.6544 - sequential_1_loss: 0.3256
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.2361 - sequential_1_loss: 0.4590
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.9319 - sequential_1_loss: 0.1811
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.6310 - sequential_1_loss: 0.2263
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.1940 - sequential_1_loss: 0.5255
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.2445 - sequential_1_loss: 0.3051
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.2596 - sequential_1_loss: 0.4850
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.2179 - sequential_1_loss: 0.5073
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8466 - sequential_1_loss: 0.2607
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.0216 - sequential_1_loss: 0.4197
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.8722 - sequential_1_loss: 0.3000
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.1223 - sequential_1_loss: 0.5366
Epoch 1/1
32/32 [=============================

Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.0922 - sequential_1_loss: 0.2834
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 1.1819 - sequential_1_loss: 0.2985
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.7090 - sequential_1_loss: 0.1674
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.0291 - sequential_1_loss: 0.4409
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.6619 - sequential_1_loss: 0.2755
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.3360 - sequential_1_loss: 0.1570
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.9160 - sequential_1_loss: 0.4261
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 0.7853 - sequential_1_loss: 0.3199
Epoch 1/1
32/32 [==============================] - 0s 3ms/step - loss: 0.7918 - sequential_1_loss: 0.2540
Epoch 1/1
32/32 [=============================

In [18]:
X_test = load_data("D:\\Work\\Signature Verification\\")

D:\Work\Signature Verification\001_1.png
D:\Work\Signature Verification\002_5.png
D:\Work\Signature Verification\003_3.png
D:\Work\Signature Verification\0100003_5.png
D:\Work\Signature Verification\0108008_4.png


In [21]:
model.save_weights("D:\\Work\\Signature Verification\\model.h5")

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 250, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100, 250, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100, 250, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          28741504    input_1[0][0]                    
                                                                 input_2[0][0]                    
          